In [1]:
import os
import torch
import spacy
import torch.nn.functional as F
from predict import check_special_token
from transformers import BertConfig, BertForMaskedLM, BertTokenizer

In [2]:
def extrapolate(keyword_list):
    result=[]
    result.append('[MASK]')
    for keyword in keyword_list:
        result.append(keyword)
        result.append('[MASK]')
    return result

def padding(token_embedding_id_list, segment_embedding_list, attention_embedding_list):
    
    max_len=361
    while len(token_embedding_id_list) < max_len:
        token_embedding_id_list.append(0)
        segment_embedding_list.append(0)
        attention_embedding_list.append(0)
    assert len(token_embedding_id_list)==len(segment_embedding_list)
    assert len(token_embedding_id_list)==len(attention_embedding_list)
    return token_embedding_id_list, segment_embedding_list, attention_embedding_list

def spacy_tokened(nlp,sentence):
    doc = nlp(sentence)
    for token in doc: 
        print(token.text,token.tag_)

In [3]:
 # load settings
tokenizer =BertTokenizer(vocab_file='bert-base-uncased-vocab.txt')
nlp =spacy.load("model/spacy/en_core_web_md-2.3.1/en_core_web_md/en_core_web_md-2.3.1")
 


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [24]:
input_semantic_sentence="yes . - why would he tip his hand like that ? "
input_syntactic_keyword=['why','he','you','a','tip']

input_sentence=[]
input_semantic=[]
tokenized_text=tokenizer.tokenize(input_semantic_sentence)
input_semantic.append("[CLS]")
input_semantic.extend(tokenized_text)
input_semantic.append("[SEP]")


syntactic_iteration=extrapolate(input_syntactic_keyword)
while check_special_token(syntactic_iteration):

    input_sentence.extend(input_semantic)
    input_sentence.extend(syntactic_iteration.copy())
    
    #segment_embedding
    input_segment=[]
    SEP_flag=False
    for token in input_sentence:  
        if (SEP_flag):
            input_segment.append(1)
        else:
            if token != "[SEP]":
                input_segment.append(0) 
            elif token=="[SEP]" :
                SEP_flag=True
                input_segment.append(0) 
    # print(input_segment)

    #attention_embedding
    input_attention=[]
    for k in range(len(input_sentence)):
        input_attention.append(1)  # [CLS]
    # print(input_attention)
    
    
    if len(input_sentence)>512:
        break
    syntactic_iteration.clear()
    
    input_sentence_ids=tokenizer.convert_tokens_to_ids(input_sentence)
    
    # padding
#     input_sentence_ids,input_segment,input_attention=padding(input_sentence_ids,input_segment,input_attention)

    
    tokens_tensor=torch.tensor([input_sentence_ids])
    segment_tensor=torch.tensor([input_segment])
    attention_tensor=torch.tensor([input_attention])
    outputs=model(input_ids=tokens_tensor)
#     outputs=model(input_ids=tokens_tensor, token_type_ids=segment_tensor,attention_mask=attention_tensor)
    predictions=outputs[0]
    output_sentence=[]
    syn_flag=False
    for word in input_sentence: 
        if (syn_flag) and word !="[MASK]":
            # output_sentence.append(word)
            syntactic_iteration.append(word)
        if word =='[MASK]':
            maskpos=input_sentence.index('[MASK]')
            logit_prob = F.softmax(predictions[0, maskpos]).data.tolist()
            predicted_index = torch.argmax(predictions[0, maskpos]).item()
            predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
            # print(predicted_token,logit_prob[predicted_index])
            # output_sentence.append(predicted_token)
            input_sentence[maskpos]=predicted_token
            if predicted_token!='[SEP]' and predicted_token!='[UNK]':
                syntactic_iteration.append('[MASK]')
                syntactic_iteration.append(predicted_token)
                syntactic_iteration.append('[MASK]')
        if (not syn_flag) and (word=="[SEP]"):
            syn_flag=True
    # print(syntactic_iteration)
    input_sentence.clear()
output_sentence=syntactic_iteration

print(input_semantic)
print(input_syntactic_keyword)
print(output_sentence)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


['[CLS]', 'yes', '.', '-', 'why', 'would', 'he', 'tip', 'his', 'hand', 'like', 'that', '?', '[SEP]']
['why', 'he', 'you', 'a', 'tip']
['why', 'would', 'he', 'give', 'you', 'a', 'tip']


In [5]:
spacy_tokened(nlp,input_semantic_sentence)

i PRP
just RB
need VBP
a DT
word NN
with IN
stein NNP
. .


In [2]:
accepted_pos_list=["WRB","WP","WP$","WDT","IN","TO","MD"]
from data_files_preprocess import check_syntactic_sentence
sentence="marina , we're going up .\n"
print(check_syntactic_sentence(sentence,accepted_pos_list))

True
